## Install Packages to Extract Data

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

!pip install pyunpack
!pip install patool
os.system('apt-get install p7zip')


from pyunpack import Archive
import shutil



## Create Dir to store data

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


#Archive('/kaggle/input/favorita-grocery-sales-forecasting/').extractall('/kaggle/working/')
for dirname, _, filenames in os.walk('/kaggle/input/'):
    for filename in filenames:
        if not os.path.exists('/kaggle/working/'+filename.split(".")[0]+"/"):
            os.makedirs('/kaggle/working/'+filename.split(".")[0]+"/")
        #print(os.path.join(dirname, filename))
#        if not os.path.exists('/kaggle/working/'+filename.split(".")[0]+"/"):
            #Archive(os.path.join(dirname, filename)).extractall('/kaggle/working/'+filename.split(".")[0]+"/")
        #print(os.path.join(dirname, filename))
        #print(filename.split(".")[0])

      

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Extract All Data

In [ ]:
Archive("/kaggle/input/favorita-grocery-sales-forecasting/train.csv.7z").extractall('/kaggle/working/train')
Archive("/kaggle/input/favorita-grocery-sales-forecasting/test.csv.7z").extractall('/kaggle/working/test')
Archive("/kaggle/input/favorita-grocery-sales-forecasting/oil.csv.7z").extractall('/kaggle/working/oil')
Archive("/kaggle/input/favorita-grocery-sales-forecasting/stores.csv.7z").extractall('/kaggle/working/stores')
Archive("/kaggle/input/favorita-grocery-sales-forecasting/items.csv.7z").extractall('/kaggle/working/items')
Archive("/kaggle/input/favorita-grocery-sales-forecasting/transactions.csv.7z").extractall('/kaggle/working/transactions')
Archive("/kaggle/input/favorita-grocery-sales-forecasting/holidays_events.csv.7z").extractall('/kaggle/working/holidays_events')

## Data Preparation
1. Load Data
2. Reduce Memory
3. Merge Data

In [ ]:

dtypes = {'id':'uint32', 'item_nbr':'int32', 'store_nbr':'int8', 'onpromotion':str}
data = {
    'tra': pd.read_csv('../working/train/train.csv', dtype=dtypes, parse_dates=['date']),
    'tes': pd.read_csv('../working/test/test.csv', dtype=dtypes, parse_dates=['date']),
    'ite': pd.read_csv('../working/items/items.csv'),
    'sto': pd.read_csv('../working/stores/stores.csv'),
    'trn': pd.read_csv('../working/transactions/transactions.csv', parse_dates=['date']),
    'hol': pd.read_csv('../working/holidays_events/holidays_events.csv', dtype={'transferred':str}, parse_dates=['date']),
    'oil': pd.read_csv('../working/oil/oil.csv', parse_dates=['date']),
    }

In [ ]:
#start_mem = test.memory_usage().sum() / 1024**2
#print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))


## Top 10 selling products across geographis [In termrs of Daily Transaction] 

Deprecated: Item Number 329362 in Store Number 44

In [ ]:
#test = test.merge(right = data['ite'], on='item_nbr')
#test = test.merge(right=data['sto'],on='store_nbr')

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(15,6))
train = data['tra']
top_ten_selling_products = train.groupby(['item_nbr']).count()['id'].sort_values(ascending=False).head(10)
ax = top_ten_selling_products.plot.bar(x='item_nbr', y='id', rot=0)
plt.ylabel("Total Unit Sold")
plt.title("Top 10 Selling Products")
plt.plot()

### Top 10 Selling Product Counts

In [ ]:
top_ten_selling_products.head(10)

## Daily Trasaction Plot of top 10 Selling products

Also Save files of each product as a time series in directory


In [ ]:
plt.figure(figsize=(16,8))
#i=1
for item_nbr in top_ten_selling_products.index:
    top1 = data['tra'][data['tra'].item_nbr == item_nbr]
    #filename = str(i)+' '+str(item_nbr)+'.csv'
    #top1.to_csv(filename, index=False)
    #i=i+1
    top1.groupby(['date']).sum()['unit_sales'].rolling(window=7).mean().plot(x='date',y='unit_sales',rot=0, label=item_nbr)
    plt.legend(loc="upper right")
plt.ylabel("Total Unit Sold")
plt.title("Top 10 Selling Products")
plt.plot()

## There are three extream events 

1. [First is  between 2013 and 2014]

2. [Second is between 2015 and 2016] 

3. [Second is between 2016 and 2017] 

Out objective is to estimate extream event in 2016-17


In [ ]:
plt.figure(figsize=(15,6))
top1 = data['tra'][data['tra'].item_nbr == 265559 ]
#top1.groupby(['date']).sum()['unit_sales'].rolling(window=7).mean().plot(x='date',y='unit_sales',rot=0, label=265559, color="yellow")
#plt.plot()
top1.groupby(['date']).sum()['unit_sales']['2013'].rolling(window=7).mean().plot(x='date',y='unit_sales',rot=0,label = 2013, color='green')
plt.plot()
top1.groupby(['date']).sum()['unit_sales']['2014'].rolling(window=7).mean().plot(x='date',y='unit_sales',rot=0,label = 2014, color='blue')
plt.plot()
top1.groupby(['date']).sum()['unit_sales']['2015'].rolling(window=7).mean().plot(x='date',y='unit_sales',rot=0,label = 2015, color='red')
plt.plot()
top1.groupby(['date']).sum()['unit_sales']['2016'].rolling(window=7).mean().plot(x='date',y='unit_sales',rot=0,label = 2016, color='orange')
plt.plot()
top1.groupby(['date']).sum()['unit_sales']['2017'].rolling(window=7).mean().plot(x='date',y='unit_sales',rot=0,label = 2017, color='brown')
plt.plot()

plt.ylabel("Total Unit Sold")
plt.title("Year Wise Products Sale")
plt.legend(loc="upper right")

# Using statistical method for estimation

#### Prepare time Series for estimation

In [ ]:
unit_sales_by_date = top1.groupby('date').sum()['unit_sales']

### Import Evaluation Metrics

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

### Adfuller Test to check stationary time series

In [ ]:
from statsmodels.tsa.stattools import adfuller
X = unit_sales_by_date.values
result = adfuller(X)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

if result[0] < result[4]["5%"]:
    print ("Reject Ho - Time Series is Stationary")
else:
    print ("Failed to Reject Ho - Time Series is Non-Stationary")

## Check ACF and PACF to decide model order

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.graphics.tsaplots import plot_acf
plot_pacf(unit_sales_by_date)
plot_acf(unit_sales_by_date, lags=7)


## Moving Average Model

In [ ]:
def moving_average_forecast(series, window_size):
    """Forecasts the mean of the last few values.
     If window_size=1, then this is equivalent to naive forecast"""
    forecast = []
    for time in range(len(series) - window_size):
        forecast.append(series[time:time + window_size].mean())
    return np.array(forecast)
split_time = 1455
moving_average_days = 6
moving_avg = moving_average_forecast(unit_sales_by_date,moving_average_days )# [split_time + 7:]

print(moving_avg.shape,unit_sales_by_date.shape)

print("mean_squared_error",mean_squared_error(unit_sales_by_date.values[moving_average_days:], moving_avg))
print("mean_absolute_error",mean_absolute_error(unit_sales_by_date.values[moving_average_days:], moving_avg))
print("mean_absolute_percentage_error",mean_absolute_percentage_error(unit_sales_by_date.values[moving_average_days:], moving_avg))

# Plot Moving average vs Actual
plt.figure(figsize=(15,6))

plt.plot(unit_sales_by_date.values[moving_average_days:], label="Actual")
#plt.plot(moving_avg[1455:], label="Test Data", color="Red")
plt.plot(moving_avg, label="Moving average")
plt.ylabel("Total Unit Sold")
plt.xlabel("Day")
plt.title("Moving Average Forecast")
plt.legend(loc="upper right")

## AR Model 

In [ ]:
x_train = unit_sales_by_date['2013':'2016'].values
x_test  = unit_sales_by_date['2017'].values

df = pd.DataFrame()

df["Original Values"]  = unit_sales_by_date
df["shift1"] = df["Original Values"].shift()
df["shift2"] = df["shift1"].shift()
df["shift3"] = df["shift2"].shift()
df["shift4"] = df["shift3"].shift()
df["shift5"] = df["shift4"].shift()
df["shift6"] = df["shift5"].shift()
#df["shift7"] = df["shift6"].shift()
#df["shift8"] = df["shift7"].shift()
lag_value = 6
df.dropna(inplace=True)
#df.head(10)

x_train, y_train = df['2013':'2016'].iloc[:,0:lag_value].values, df['2013':'2016'].iloc[:,lag_value:].values
x_test, y_test = df['2017'].iloc[:,0:lag_value].values, df['2017'].iloc[:,lag_value:].values

from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(x_train, y_train)
print(reg.coef_)
print(reg.intercept_)


ar_predictions = reg.predict(x_test)

plt.figure(figsize=(15,6))
plt.plot(ar_predictions ,label = "Predictions")
plt.plot(y_test, label = "Original" )
plt.xlabel("Days")
plt.ylabel("Sale Count")

print("mean_squared_error",mean_squared_error(y_test, ar_predictions))
print("mean_absolute_error",mean_absolute_error(y_test, ar_predictions))
print("mean_absolute_percentage_error",mean_absolute_percentage_error(y_test, ar_predictions))

#ar_predictions = reg.predict(x_train)
#plt.figure(figsize=(15,6))
#plt.plot(ar_predictions,label = "Predictions")
#plt.plot(y_test, label = "Y_train")
plt.legend(loc="upper right")
#print("mean_squared_error",mean_squared_error(y_test, ar_predictions))
#print("mean_absolute_error",mean_absolute_error(y_test, ar_predictions))
#print("mean_absolute_percentage_error",mean_absolute_percentage_error(y_test, ar_predictions))

## ARMA Model

In [ ]:
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARMA
from datetime import datetime, timedelta

x_train, y_train = df['2013':'2016'].iloc[:,0:lag_value], df['2013':'2016'].iloc[:,lag_value:]
x_test, y_test = df['2017'].iloc[:,0:lag_value], df['2017'].iloc[:,lag_value:]

ARMA_model = ARMA(x_train.values.reshape(-1).tolist(), (6,6)).fit()
arma_predictions = ARMA_model.predict(start=len(x_train), end=len(x_train) + len(x_test)-1, dynamic=False)


plt.figure(figsize=(15,6))
plt.plot(arma_predictions ,label = "Predictions")
plt.plot(y_test.values, label = "Original" )
plt.xlabel("Days")
plt.ylabel("Sale Count")
plt.legend(loc="upper right")

print("mean_squared_error",mean_squared_error(y_test, arma_predictions))
print("mean_absolute_error",mean_absolute_error(y_test, arma_predictions))
print("mean_absolute_percentage_error",mean_absolute_percentage_error(y_test, arma_predictions))


In [ ]:
'''
prediction_rolling = unit_sales_by_date
for end_date in x_test.index:
    train_data = unit_sales_by_date[:end_date-timedelta(days=1)]
    print(end_date,train_data.shape)
    model = ARMA(train_data, (6,6)).fit()

    
    #model_fit = model.fit()
    #pred = model_fit.predict(end_data)
    #prediction_rolling.loc[end_date] = pred.loc[end_date]
#prediction_rolling.head()
'''
#print("mean_squared_error",mean_squared_error(x_test, predictions))
#print("mean_absolute_error",mean_absolute_error(x_test, predictions))
#print("mean_absolute_percentage_error",mean_absolute_percentage_error(x_test, predictions))

## ARIMA model

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

model = ARIMA(x_train.values.reshape(-1).tolist(), order=(6,2,6)).fit()

arima_predictions = []

arima_predictions = model.predict(start=len(x_train),end=len(x_train) + len(x_test)-1,dynamic=False)
#evaluate(predictions, x_test)

print("mean_squared_error",mean_squared_error(y_test, arima_predictions))
print("mean_absolute_error",mean_absolute_error(y_test, arima_predictions))
print("mean_absolute_percentage_error",mean_absolute_percentage_error(y_test, arima_predictions))

plt.figure(figsize=(15,6))
plt.plot(arima_predictions ,label = "Predictions")
plt.plot(y_test.values, label = "Original" )
plt.xlabel("Days")
plt.ylabel("Sale Count")
plt.legend(loc="upper right")